In [1]:
#!pip install scikit-learn

In [27]:
#!pip install numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 6.1 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 7.7 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 9.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1


In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
from sklearn.preprocessing import TargetEncoder
import gc

import numba


In [2]:
#a = pd.DataFrame({'a':[1,2,3], 'b':[3,4,5], 'c':[7,8,9]})
#a

In [3]:
#a = pd.DataFrame({'a':[1,2,3], 'b':[3,4,5], 'c':[7,8,9]})
#a = a[['c','b','a']]
#a.ewm(alpha=0.7, min_periods=0, axis=1).mean(engine="numba")

In [4]:
import sys

def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)
    
   

In [5]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                              locals().items())), key= lambda x: -x[1])[:10]:
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                 TargetEncoder:  1.7 KiB
                            _i:  384.0 B
                           _i4:  384.0 B
                           _i5:  260.0 B
                           _ii:  183.0 B
                           _i3:  183.0 B
                           _i1:  174.0 B
                          open:  160.0 B
                    sizeof_fmt:  160.0 B
                           _ih:  120.0 B


In [6]:
#numba.set_num_threads(5)

In [7]:
#z=a.T.ewm(alpha=0.7, min_periods=0).mean(engine="numba").iloc[:,-1];

In [8]:
data_train = pd.read_csv('../data_cleaned/data_train.csv')
item_categories = pd.read_csv('../data_cleaned/item_categories.csv')
items = pd.read_csv('../data_cleaned/items.csv')
shops = pd.read_csv('../data_cleaned/shops.csv')

In [9]:
#merged = data_train.merge(items, how='left').merge(item_categories, how='left').merge(shops, how='left')

In [10]:
def parse_city(shop_name):
    if shop_name.split()[0] == '!Якутск':
        return  'Якутск'

    if shop_name.split()[0] == 'Сергиев':
            return  'Сергиев Посад'
    else:
        return shop_name.split()[0]

In [11]:
#dates = pd.to_datetime(data_train['date'], format='%d.%m.%Y')


In [12]:
import calendar

def number_to_month(numbers):
    # Create a list of month names using the calendar module
    month_names = list(calendar.month_name)[1:]  # Extract months from 1 to 12 (exclude the empty element at index 0)
    
    # Convert the numbers to month names
    result = [month_names[num % 12] for num in numbers]  # Use num % 12 to handle numbers beyond 11
    
    return result


In [13]:

def number_to_month(numbers):
    result = [num %12 for num in numbers]  # Use num % 12 to handle numbers beyond 11
    
    return result


In [14]:
def convert_to_year(month_num):
    # January 2013 is the starting point
    start_year = 2013
    
    # Calculate the year
    year = start_year + (month_num // 12)
    
    return year

In [15]:
a= pd.DataFrame({'a':[1,2,3]})
b=a.copy()
a['a'] +=1
print(b)

   a
0  1
1  2
2  3


In [16]:

def create_lags(data,item_lags=None,price_lags=None, test_included=False):
    #if test_included:
    #    lagged_items = data[data['date_block_num'] < 34] [['date_block_num','item_cnt_month','shop_id','item_id']].copy()
    #    lagged_prices = data[data['date_block_num'] < 34] [['date_block_num','avg_item_price','shop_id','item_id']].copy()
    #else:
    #    lagged_items = data[['date_block_num','item_cnt_month','shop_id','item_id']].copy()
    #    lagged_prices = data[['date_block_num','avg_item_price','shop_id','item_id']].copy()
    lagged_items = data[['date_block_num','item_cnt_month','shop_id','item_id']]
    lagged_prices = data[['date_block_num','avg_item_price','shop_id','item_id']]
    for lag in item_lags:
        lagged_items['date_block_num']+=lag#previous month becomes present
        data=data.merge(lagged_items, how='left',on=['date_block_num','shop_id','item_id'], suffixes=('', f'_lag_{lag}'))
        lagged_items['date_block_num']-=lag

    
    for lag in price_lags:
        lagged_prices['date_block_num']+=lag#previous month becomes present
        data=data.merge(lagged_prices, how='left',on=['date_block_num','shop_id','item_id'], suffixes=('', f'_lag_{lag}'))
        lagged_prices['date_block_num']-=lag

        
    return data

In [17]:
def find_mean_price(lagged):
    return lagged[[f'avg_item_price_lag_{lag}' for lag in price_lags[:6]]].mean(axis=1)
    

In [18]:
def find_mean_cnt(lagged):
    return lagged[[f'item_cnt_month_lag_{lag}' for lag in item_lags[:6]]].mean(axis=1)
    

In [19]:
def avg(data):
    v = data['item_cnt_month'].mode().mean()

    return v#.mode()[0]#.mean()

In [20]:
def avg_by_city(data, city):
    v = data [data['city']==city] ['item_cnt_month'].mode().mean()
    
    return v# .mode()[0]

In [21]:
def avg_by_shop(data, shop):
    v = data [data['shop_id']==shop] ['item_cnt_month'].mode().mean()
    
    return v# .mode()[0]#.mean()

In [22]:

def create_lags_columns(data, columns,item_lags=[1,2,3,4,5,6],price_lags=[1,2,3,4,5,6], test_included=False):#date_block_num required as first argument
    #if test_included:
    #    lagged_items = data [data['date_block_num'] < 34]  [[*columns,'item_cnt_month']] 
    #    lagged_prices = data [data['date_block_num'] < 34]  [[*columns,'avg_item_price']]
    #else:
    lagged_items = data[[*columns,'item_cnt_month']]
    lagged_prices = data[[*columns,'avg_item_price']]
    
    for lag in item_lags:
        lagged_items['date_block_num']+=lag#previous month becomes present

        averaged = lagged_items.groupby(columns).mean().reset_index()#future 
        
        data=data.merge(averaged, how='left',on=columns, suffixes=('', f'{'_'.join(columns[1:])}_lag_{lag}'))
        
        lagged_items['date_block_num']-=lag

    
    for lag in price_lags:
        lagged_prices['date_block_num']+=lag#previous month becomes present

        averaged = lagged_prices.groupby(columns).mean().reset_index()
        
        data=data.merge(averaged, how='left',on=columns, suffixes=('', f'{'_'.join(columns[1:])}_lag_{lag}'))
        lagged_prices['date_block_num']-=lag

        
    return data

In [23]:
import pandas as pd
import numpy as np

def calculate_ema_3(df, target='item_cnt_month', columns=None, test_included=True, alpha=2/(3+1)):
    weights = (1 - alpha) ** np.arange(3)

    # Define the columns for lags
    lag_cols = [f'{target}{'_'.join(columns[1:])}_lag_{i}' for i in range(1, 4)]

    # Replace NaNs with zeros in the lags
    df_in = df[lag_cols].fillna(0)

    # Calculate weighted sums and weights
    weighted_sums = np.dot(df_in[lag_cols].values, weights)
    valid_weights = (df_in[lag_cols] != 0).values.dot(weights)

    # Avoid division by zero
    valid_weights[valid_weights == 0] = np.nan

    # Calculate EMA
    df[f'ema_3_{target}_{'_'.join(columns[1:])}'] = weighted_sums / valid_weights

    return df

# Example usage:
# df = calculate_ema_3(df, target='item_cnt_month', columns=['date_block_num'])

In [24]:
import pandas as pd
import numpy as np
#alpha == 0 is simple mean
def calculate_ema_6(df, min_periods=3, target='item_cnt_month', columns=None, test_included=True, alpha=2/(6+1)):
    weights = (1 - alpha) ** np.arange(6)
    print(weights)
    # Define the columns for lags
    lag_cols = [f'{target}{'_'.join(columns[1:])}_lag_{i}' for i in range(1, 7)]

    # Replace NaNs with zeros in the lags
    df_in = df[lag_cols].fillna(0)

    # Calculate weighted sums and weights
    weighted_sums = np.dot(df_in[lag_cols].values, weights)
    valid_weights = (df_in[lag_cols] != 0).values.dot(weights)

    # Avoid division by zero
    print(valid_weights)
    valid_weights[valid_weights == 0] = np.nan

    # Calculate EMA
    df[f'ema_6_{target}_{'_'.join(columns[1:])}'] = weighted_sums / valid_weights
    df[f'ema_6_{target}_{'_'.join(columns[1:])}']=df[f'ema_6_{target}_{'_'.join(columns[1:])}'].fillna(0)
    return df

# Example usage:
# df = calculate_ema_3(df, target='item_cnt_month', columns=['date_block_num'])

In [25]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [26]:
def create_last_seen(data):#Takes some minutes
    prevs=data.groupby(['item_id'])['date_block_num'].unique().rename('blocks')
    mer = data.merge(prevs, how='left', on='item_id')
    for idx in range(len(mer)):
        curr =  mer.loc[idx,'blocks'] 
        
        prevs = np.array(curr) < mer.loc[idx,'date_block_num']
    
        if sum(prevs) == 0:
            mer.loc[idx,'blocks'] = -1
        else:
            mer.loc[idx,'blocks'] = curr [prevs] .max()

    data['last_seen'] = mer['blocks']
    return data
            

In [27]:
"""
find mean for last : month
(item_id)
(item_id, city)
(item_id, shop_id)
(item_category_id, shop_id)
(item_category_id, city)
"""
def find_means_last6(lagged, columns=None):#date_block_num required as first argument
    print(list(lagged.columns))
    lagged=lagged.fillna(0)
    means = lagged[['item_cnt_month_lag_1', 
                 'item_cnt_month_lag_2',
                 'item_cnt_month_lag_3',
                 'item_cnt_month_lag_4', 
                 'item_cnt_month_lag_5', 
                 'item_cnt_month_lag_6'
                   ]].mean(axis=1).rename('avg')

    
   
    m_i = pd.concat([lagged[columns], means],axis=1)
    m_i = m_i.groupby(columns)['avg'].transform('mean')
    lagged[f'mean_item_cnt_6month{'_'.join(columns[1:])}'] = m_i
    return lagged


In [28]:
def create_EMAs(df,cols, test_included=True, alpha=None):
    df = create_lags_columns(df,columns=cols,test_included=test_included )
    start = time.time()
    #df = calculate_ema_3(df, target='item_cnt_month', columns=cols,test_included=test_included, alpha=alpha )
    #df = calculate_ema_3(df, target='avg_item_price', columns=cols,test_included=test_included ,alpha=alpha)
    df = calculate_ema_6(df, target='item_cnt_month', columns=cols,test_included=test_included,alpha=alpha )
    #df = calculate_ema_6(df, target='avg_item_price', columns=cols,test_included=test_included )
    end = time.time()
    #print('memory before drop', df.memory_usage())
    
    df=df.drop([f'item_cnt_month{'_'.join(cols[1:])}_lag_1', 
                f'item_cnt_month{'_'.join(cols[1:])}_lag_2', 
                f'item_cnt_month{'_'.join(cols[1:])}_lag_3',
                f'avg_item_price{'_'.join(cols[1:])}_lag_1', 
                f'avg_item_price{'_'.join(cols[1:])}_lag_2', 
                f'avg_item_price{'_'.join(cols[1:])}_lag_3',
               ], axis=1)
    
    df=df.drop([f'item_cnt_month{'_'.join(cols[1:])}_lag_4', 
                f'item_cnt_month{'_'.join(cols[1:])}_lag_5', 
                f'item_cnt_month{'_'.join(cols[1:])}_lag_6',
                f'avg_item_price{'_'.join(cols[1:])}_lag_4', 
                f'avg_item_price{'_'.join(cols[1:])}_lag_5', 
                f'avg_item_price{'_'.join(cols[1:])}_lag_6',
               ], axis=1)
    
    #print('memory after drop', df.memory_usage())
    print('item_id EMA calculated , time,s:',end-start)
    return df

In [29]:
import time

In [30]:
def transform(data_train, item_categories,items,shops, test_included=True ):
    grouped=data_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_price':'mean',
                                                                    'item_cnt_day':'sum'
                                                                    })#take some (a lot)))) ) time
    
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                              locals().items())), key= lambda x: -x[1])[:10]:
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
        
    print('1st grouping done')
    
    grouped=grouped.reset_index()
    grouped = grouped.sort_values(by='date_block_num')
    merged = grouped.merge(items, how='left').merge(item_categories, how='left').merge(shops, how='left')
    
    print('merged created')

   
    merged['month'] = np.array(number_to_month(merged['date_block_num'])).astype(np.uint8)
    merged['year'] = merged['date_block_num'].apply(convert_to_year).astype(np.uint16)
    merged['super_category'] = merged['item_category_name'].apply(lambda a: a.split()[0]).astype('category').cat.codes.astype(np.uint8)
    merged['city'] = merged['shop_name'].apply(parse_city).astype('category').cat.codes.astype(np.uint8)
    merged=merged.rename(columns={'item_price':'avg_item_price','item_cnt_day':'item_cnt_month'})
    
    
    ALPHA = 0.12
    merged['shop_id_cat'] = merged['shop_id'].astype(np.uint8)
    merged['item_id_cat'] = merged['item_id'].astype(np.uint16)
    merged['item_category_id_cat'] = merged['item_category_id'].astype(np.uint8)

    
    cols=['date_block_num','item_id']
    merged=create_EMAs(merged,cols, alpha = ALPHA)#168.3924057483673
    print(f'{cols} EMA calculated')
    
    
    cols=['date_block_num','item_id','city']#--------------
    merged=create_EMAs(merged,cols, alpha = ALPHA)
    print(f'{cols} EMA calculated')
    

    cols=['date_block_num','item_category_id_cat','shop_id']#--------------
    merged=create_EMAs(merged,cols, alpha = ALPHA)
    print(f'{cols} EMA calculated')
    
        
    cols=['date_block_num','item_category_id_cat','city']#------------34
    merged=create_EMAs(merged,cols, alpha = ALPHA)
    print(f'{cols} EMA calculated')

    #cols=['date_block_num','shop_id']#------------34
    #merged=create_EMAs(merged,cols)
    #print(f'{cols} EMA calculated')

    cols=['date_block_num','item_id','shop_id']#------------34
    merged=create_EMAs(merged,cols, alpha = 0.0)
    print(f'{cols} EMA calculated')

    #merged = create_last_seen(merged)
    print('Last seen created')
    
    item_lags=list([*range(1,7)])
    price_lags=list([*range(1,2)])
    
    merged = create_lags(merged,item_lags=item_lags,price_lags=price_lags,test_included=test_included )#takes a bit time
    
    merged = create_lags_columns(merged, columns=['date_block_num','item_id'],item_lags=item_lags,price_lags=price_lags, test_included=False)
    merged = create_lags_columns(merged,  columns=['date_block_num','shop_id'],item_lags=item_lags,price_lags=price_lags, test_included=False)
    merged = create_lags_columns(merged, columns=['date_block_num','item_category_id_cat','shop_id'],item_lags=item_lags,price_lags=price_lags, test_included=False)
    merged = create_lags_columns(merged,  columns=['date_block_num','item_category_id_cat'],item_lags=item_lags,price_lags=price_lags, test_included=False)
    
    print('laggs created')
    
    
    
   # merged = find_means_last6(merged, ['date_block_num','item_id','shop_id',])
   # merged = find_means_last6(merged, ['date_block_num','item_id'])
   # merged = find_means_last6(merged, ['date_block_num','item_category_id','shop_id'])
   # merged = find_means_last6(merged, ['date_block_num','item_category_id','city'])
   # merged = find_means_last6(merged, ['date_block_num','item_category_id'])
   

    
    
    print('means for last 6 mothes found')
    
    merged['first_date_block_num'] = merged.groupby('item_id')['date_block_num'].transform('min')
    merged['date_block_num_diff'] = merged['date_block_num'] - merged['first_date_block_num']
    
    
    
    return merged#merged['date_block_num' >= 3] - for lag-based learning()

In [31]:
#data_train.memory_usage() / (10**6)

In [32]:
test = pd.read_csv('../data_cleaned/test.csv')

In [33]:
test['date_block_num'] = 34

In [34]:
transformed = transform(pd.concat([data_train,test]),  item_categories,items,shops,test_included=True )

                    data_train: 339.2 MiB
                       grouped: 34.9 MiB
                         items:  3.3 MiB
               item_categories:  9.9 KiB
                         shops:  6.9 KiB
                 test_included:   28.0 B
1st grouping done
merged created


/tmp/ipykernel_13184/434146873.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']+=lag#previous month becomes present
/tmp/ipykernel_13184/434146873.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']-=lag
/tmp/ipykernel_13184/434146873.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

[1.         0.88       0.7744     0.681472   0.59969536 0.52773192]
[0.         0.         0.         ... 4.46329928 4.46329928 4.46329928]
item_id EMA calculated , time,s: 0.28055763244628906
['date_block_num', 'item_id'] EMA calculated


/tmp/ipykernel_13184/434146873.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']+=lag#previous month becomes present
/tmp/ipykernel_13184/434146873.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']-=lag
/tmp/ipykernel_13184/434146873.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

[1.         0.88       0.7744     0.681472   0.59969536 0.52773192]
[0.         0.         0.         ... 2.08920392 0.         0.        ]
item_id EMA calculated , time,s: 0.16550207138061523
['date_block_num', 'item_id', 'city'] EMA calculated


...

/tmp/ipykernel_13184/434146873.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']+=lag#previous month becomes present
/tmp/ipykernel_13184/434146873.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']-=lag
/tmp/ipykernel_13184/434146873.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

[1. 1. 1. 1. 1. 1.]
[0. 0. 0. ... 3. 0. 0.]
item_id EMA calculated , time,s: 0.4187917709350586
['date_block_num', 'item_id', 'shop_id'] EMA calculated
Last seen created


/tmp/ipykernel_13184/1466009272.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']+=lag#previous month becomes present
/tmp/ipykernel_13184/1466009272.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']-=lag
/tmp/ipykernel_13184/1466009272.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

laggs created
means for last 6 mothes found


/tmp/ipykernel_13184/434146873.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_prices['date_block_num']-=lag


In [35]:
test_trans=transformed

In [36]:
list(test_trans.columns)


['shop_id',
 'item_id',
 'date_block_num',
 'avg_item_price',
 'item_cnt_month',
 'item_name',
 'item_category_id',
 'item_category_name',
 'shop_name',
 'month',
 'year',
 'super_category',
 'city',
 'shop_id_cat',
 'item_id_cat',
 'item_category_id_cat',
 'ema_6_item_cnt_month_item_id',
 'ema_6_item_cnt_month_item_id_city',
 'ema_6_item_cnt_month_item_category_id_cat_shop_id',
 'ema_6_item_cnt_month_item_category_id_cat_city',
 'ema_6_item_cnt_month_item_id_shop_id',
 'item_cnt_month_lag_1',
 'item_cnt_month_lag_2',
 'item_cnt_month_lag_3',
 'item_cnt_month_lag_4',
 'item_cnt_month_lag_5',
 'item_cnt_month_lag_6',
 'avg_item_price_lag_1',
 'item_cnt_monthitem_id_lag_1',
 'item_cnt_monthitem_id_lag_2',
 'item_cnt_monthitem_id_lag_3',
 'item_cnt_monthitem_id_lag_4',
 'item_cnt_monthitem_id_lag_5',
 'item_cnt_monthitem_id_lag_6',
 'avg_item_priceitem_id_lag_1',
 'item_cnt_monthshop_id_lag_1',
 'item_cnt_monthshop_id_lag_2',
 'item_cnt_monthshop_id_lag_3',
 'item_cnt_monthshop_id_lag_4',

In [37]:
test_trans.to_csv('../data.csv')

In [29]:
#set_y = set(map(tuple, test_trans.query('date_block_num==33') [['shop_id','item_id']].values))
#idx = [tuple(point) not in set_y for point in test_trans.query('date_block_num==34') [['shop_id','item_id']].values]

#test_trans.query('date_block_num==34') [['shop_id','item_id']].values[idx].shape

In [27]:
!pwd

/home/valeriy/python_projects/predict_future_sales/EDA
